In [65]:
import re
import pandas as pd

ratings_data = pd.read_csv('./data/movies_data/ratings_movies.csv', sep=',')

In [66]:
ratings_df = ratings_data.copy()

In [67]:
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [70]:
get_year_release('(2000)')

2000

In [69]:
ratings_df['year_release'] = ratings_df['title'].apply(get_year_release)

ratings_df['year_release'].head(1000)

0      1995.0
1      1995.0
2      1995.0
3      1995.0
4      1995.0
        ...  
995    2004.0
996    2004.0
997    2002.0
998    2004.0
999    2004.0
Name: year_release, Length: 1000, dtype: float64

In [71]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [75]:
ratings_df['year_release'].astype('int32', errors='ignore')

0         1995.0
1         1995.0
2         1995.0
3         1995.0
4         1995.0
           ...  
100831    2017.0
100832    2017.0
100833    2017.0
100834    2017.0
100835    2017.0
Name: year_release, Length: 100836, dtype: float64

In [81]:
ratings_df['year_release'] = ratings_df['year_release'].astype('int32', errors='ignore', copy=False)

In [108]:
ratings_df.groupby(by=['userId', 'genres'])['genres'].count().sort_values(ascending=False)

userId  genres                             
474     Drama                                  319
448     Comedy                                 271
414     Drama                                  257
        Comedy                                 238
599     Comedy                                 226
                                              ... 
297     Crime|Thriller                           1
        Crime|Horror|Thriller                    1
64      Action|Adventure|Comedy|Thriller         1
297     Crime|Drama|Romance|Thriller             1
313     Adventure|Animation|Children|Sci-Fi      1
Name: genres, Length: 45382, dtype: int64

In [97]:
ratings_df[(ratings_df['title'] == 'Five Senses, The (1999)')]

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
17036,17036,108,3795,5.0,2003-01-17 21:56:42,"Five Senses, The (1999)",Drama,1999.0


In [117]:
ratings_df.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[False, True])

,count,mean
userId,,
414,2698,3.391957
599,2478,2.642050
474,2108,3.398956
448,1864,2.847371
274,1346,3.235884
...,...,...
278,20,3.875000
569,20,4.000000
189,20,4.100000


In [122]:
v = ratings_df[(ratings_df['year_release'] == 2018)].groupby(by='genres')['rating'].agg(['mean', 'count'])
v[(v['count']>10)]

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [124]:
def set_year_rating(arg):
    candidates = re.findall(r'\d{4}', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
#display(set_year_rating('2000-07-30 18:45:03'))

2000

In [125]:

ratings_df['year_rating'] = ratings_df['date'].apply(set_year_rating)

ratings_df['year_rating']

0         2000
1         2000
2         2000
3         2000
4         2000
          ... 
100831    2017
100832    2017
100833    2017
100834    2017
100835    2017
Name: year_rating, Length: 100836, dtype: int64

In [138]:
ratings_df.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres',
    fill_value=0,
    aggfunc='mean'
)['Comedy']

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64